In [1]:
# initialize

from registration import initial_registration
import numpy as np
import csv
import os
import sys
from pathlib import Path
import tifffile
import roifile
import numpy as np
from signal_analysis import *
from create_kymograph import create_tracking_kymo, create_kymograph, create_kymo_movie
from fit_to_kymo import track_nonmoving_particles
from glob import glob

In [2]:
# register raw data files using Caiman
# 
# Open the channel 1 data in ImageJ/FIJI.
# Make a maximum-value z-projection of ch1 data.
# Use the brightness and contrast table to check for a threshold value for the data.
# Any frames of Ch1 that have pixel values higher than this threshold are removed from the data before motion correction.'
# A good threshold will be lower than the streak values and higher than all the values for true data.
# Picking too low of a value (especially with a soma within view) will result in an empty Ch1 stack.

sys.path.append('/Users/benjaminscholl/Documents/autophagy_tal/autophagy_code') # change to wherever autophagy_tal is located

ch1_thresh = input("Ch1 upper threshold (default is 70):")
if ch1_thresh == '':
    ch1_thresh = 70
else:
    ch1_thresh = int(ch1_thresh)

wk_dir = '/Users/benjaminscholl/Documents/autophagy_tal/file_22'
initial_registration(wk_dir, ch1_thresh)

pathname = Path(wk_dir)
cond_name = pathname.parent.name
cond_subname = pathname.name 


The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman
In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_legacy will default to False (and eventually will be removed)
100%|██████████| 1/1 [00:00<00:00, 23.89it/s]
In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_legacy will default to False (and eventually will be removed)
100%|██████████| 1/1 [00:00<00:00, 23.31it/s]
In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_legacy will default to False (and eventually will be removed)
100%|██████████| 1/1 [00:00<00:00, 21.58it/s]
In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_legacy will default to False (and eventually will be removed)
100%|██████████| 1/1 [00:

In [3]:
# Check quality of registration in ImageJ/FIJI
# If quality is good, draw segmented line ROIs for dendrites/axons in Ch2 nonrigid data.
# These are the the regions that will be tracked.

# Create kymographs for tracking particles.
# One set of kymographs will be created for each ROI drawn

kymo_width = 9 # how many pixels wide to include around dendrite/axon
which_kymo = 1 # 1 means use rigid reg, 2 for nonrigid

print(f"Making kymos for: {wk_dir}")

if which_kymo == 1:
    ch1 = tifffile.imread('ch1_rigid_registered.tif')
    ch2 = tifffile.imread('ch2_rigid_registered.tif')
elif which_kymo == 2:
    ch1 = tifffile.imread('ch1_nonrigid_registered.tif')
    ch2 = tifffile.imread('ch2_nonrigid_registered.tif')
single_roifile = glob('*.roi')
if 'RoiSet.zip' in os.listdir():
    roi_list = roifile.roiread('RoiSet.zip')
elif any(single_roifile):
    roi_list = roifile.roiread(single_roifile[0])
    roi_list = [roi_list]
for i in range(len(roi_list)):
    roi_coords = roi_list[i].coordinates()
    ch1_kymo, _ = create_kymograph(ch1, roi_coords, kymo_width)
    ch2_kymo, ch2_kymo_movie = create_kymograph(ch2, roi_coords, kymo_width)
    ch2_tracking_kymo = create_tracking_kymo(ch2_kymo, roi_coords)

    if not os.path.exists('kymographs'):
        os.makedirs('kymographs')
    
    tifffile.imwrite(os.path.join('kymographs',f'ch1_kymo_roi_{i}.tif'), ch1_kymo)
    tifffile.imwrite(os.path.join('kymographs' ,f'ch2_kymo_roi_{i}.tif'), ch2_kymo)
    tifffile.imwrite(os.path.join('kymographs' ,f'ch2_tracking_kymo_roi_{i}.tif'), ch2_tracking_kymo)
    print(ch2_kymo_movie.shape)
    create_kymo_movie(ch2_kymo_movie, f'ch2_kymo_mov_roi_{i}.mp4')

Making kymos for: /Users/benjaminscholl/Documents/autophagy_tal/file_22
(9, 119, 200)
Kymograph video saved as ch2_kymo_mov_roi_0.mp4
(9, 31, 200)
Kymograph video saved as ch2_kymo_mov_roi_1.mp4
(9, 27, 200)
Kymograph video saved as ch2_kymo_mov_roi_2.mp4
(9, 34, 200)
Kymograph video saved as ch2_kymo_mov_roi_3.mp4
(9, 22, 200)
Kymograph video saved as ch2_kymo_mov_roi_4.mp4
(9, 31, 200)
Kymograph video saved as ch2_kymo_mov_roi_5.mp4
(9, 43, 200)
Kymograph video saved as ch2_kymo_mov_roi_6.mp4


In [ ]:
# grab moving particles
# 
# first need to:
# open each kymograph in ImageJ/FIJI
# Use the multi-point tool to draw a line along a 'tracking_kymo_roi'
# draw as many sets of points as there are particles in the kymograph
# save ROI set as 'trackpoints_roi_XXX', then can run this cell

all_particle_results = []

single_roifile = glob('*.roi')
if any(single_roifile):
    single_roi = roifile.roiread(single_roifile[0])
    roi_list = [single_roi]
else:
    roi_list = roifile.roiread('RoiSet.zip')
for i in range(len(roi_list)):
    roi_coords = roi_list[i].coordinates()
    dx = np.diff(roi_coords[:, 1])
    dy = np.diff(roi_coords[:, 0])
    dendrite_roi_length = np.hypot(dx, dy).sum()
    ch2_tracking_kymo = tifffile.imread(os.path.join('kymographs', f'ch2_tracking_kymo_roi_{i}.tif'))
    num_frames = ch2_tracking_kymo.shape[0]
    trackpoints_filename = glob(f"kymographs/trackpoints_roi_{i}*")

    if len(trackpoints_filename) == 0: # No particles here
        continue
    trackpoints_file = roifile.roiread(trackpoints_filename[0])

    if not isinstance(trackpoints_file, list):
        trackpoints_file = [trackpoints_file]

    for j in range(len(trackpoints_file)):
        particle = trackpoints_file[j]
        coords = particle.coordinates().astype('int')
        positions = trackpoints_to_position(coords, num_frames=num_frames)
        movement_summary = characterize_motion_from_coords(coords=coords)
        movement_summary.insert(0, dendrite_roi_length)
        movement_summary.insert(0, i) # which dendrite
        movement_summary.insert(0, cond_subname)
        movement_summary.insert(0, cond_name)
        all_particle_results.append(movement_summary)




In [ ]:
# nonmoving particles

# params for finding the peaks of nonmoving particles
prominence = 1.0 # most important parameter adjusting for identifying particles that are not moving
height = 0.0 # Ch2 must be brighter than Ch1
width = 0.5 # little effect

nonmoving_results = track_nonmoving_particles(prominence, width, height, verbose=True)
for particle in nonmoving_results:
    particle.insert(0, cond_subname)
    particle.insert(0, cond_name)
    all_particle_results.append(particle)

In [ ]:
#pull all data together and save
# each row is a particle, columns are:
#   - condition name
#   - condition subname (e.g. which acquisition number)
#   - source dendrite number
#   - dendrite mask length (pixels)
#   - proportion of tracked time moving
#   - proportion of tracked time still
#   - average speed (pixels/frame)
#   - total pixel distance moved by particle over tracking time
#   - number of times movement direction changed


data_outname = 'particle_summary_data.csv' # change to whatever you want, saved in main file

with open(data_outname, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(all_particle_results)
for row in all_particle_results:
    print(row)
print(f'Tracking results saved for {cond_name}, {cond_subname}')